#### 사용자 정의 모델 클래스
- 부모 클래스: nn.module
- 필수 오버라이딩
    - __init__(): 모델 구성 즉, 설계
    - forward(): 순방향 학습 진행 코드 구현
- 동적 모델
    - container 모듈 중 nn.ModulList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이에 연관성 없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [2]:
# 모듈로딩
import torch                                            # 텐서 관련 모듈
import torch.nn as nn                                   # 인공신경망 관련 모듈
import torch.nn.functional as F                         # 인공신경망 관련 함수들 모듈(손실,활성화 함수)
import torch.optim as optim                             # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary                           # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *                   # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *               # 분류 성능 지표 관련 모듈



- [기본] 신경망클래스  <hr>
    - 입력층 -> 입력: 피쳐수 고정
    - 출력층 -> 출력: 타겟수 고정
    - 은닉층 -> 고정
    

In [3]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    - 목표: 은닉층의 개수가 동적인 모델
    - 조건
        - 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        - 은닉층의 개수 동적 => 퍼셉트론 개수 고정 => 은닉층의 개수, 퍼셉트론 수 
     


In [4]:
# 모델이름: DynamicModel
# 부모클래스: nn.Module
# 매개변수: in_in, out_out, h_cnt

In [36]:
class DynamicModel(nn.Module):
    # 생성자 메서드
    def __init__(self, in_in, out_out, h_cnt, h_inout) -> None:
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_inout)
        self.h_layers=nn.ModuleList([nn.Linear(h_inout, h_inout) for _ in range(h_cnt)])
        self.out_layer=nn.Linear(h_inout, out_out)

    # 학습 진행 콜백 메서드
    def forward(self, x):
        y=self.in_layer(x)
        y=F.relu(y)
        
        for linear in self.h_layers:
            y= linear(y)
            y=F.relu(y)
        
        return self.out_layer(y)

In [41]:
# 모델 인스턴스 생성
m1=DynamicModel(3,1,5,10)

In [42]:
# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name, param)

in_layer.weight Parameter containing:
tensor([[-0.0375, -0.0608, -0.2472],
        [-0.3788,  0.0744,  0.0341],
        [-0.5093,  0.5015,  0.2614],
        [ 0.3550,  0.1704, -0.1549],
        [ 0.1970,  0.2679,  0.2267],
        [-0.1871, -0.3020, -0.0210],
        [ 0.4561, -0.3151, -0.2322],
        [-0.2855, -0.4827,  0.2227],
        [-0.0830,  0.0576,  0.5631],
        [ 0.0916,  0.4499,  0.0932]], requires_grad=True)
in_layer.bias Parameter containing:
tensor([-0.0614,  0.3518,  0.1764, -0.0674,  0.0194, -0.2808, -0.0461,  0.3831,
         0.4893,  0.5314], requires_grad=True)
h_layers.0.weight Parameter containing:
tensor([[-0.2307, -0.0084,  0.1268,  0.2148, -0.1745, -0.2521,  0.1073, -0.1819,
          0.3157, -0.1979],
        [ 0.1557, -0.0974, -0.2599, -0.3083,  0.1373, -0.0964, -0.2035,  0.3121,
          0.1855,  0.2082],
        [-0.1421, -0.0157,  0.0700, -0.0364,  0.1519, -0.0769, -0.0707, -0.0331,
         -0.1245, -0.2168],
        [-0.0355, -0.1131, -0.0999, -0.13

- 학습진행


In [43]:
# 임시 데이터 생성
dataTS= torch.FloatTensor(([1,3,5], [2,4,6], [3,5,7], [4,6,8]))
targetTS= torch.FloatTensor([[7,9], [8,10], [9,11], [10,12]])

In [44]:
#  모델 학습 
pre_y=m1(dataTS)
print(pre_y)

tensor([[-0.2619],
        [-0.2620],
        [-0.2618],
        [-0.2612]], grad_fn=<AddmmBackward0>)
